In [25]:
import pandas as pd
file_names = {
    "primary_cohort": "data/s41598-020-73558-3_sepsis_survival_primary_cohort.csv",
    "study_cohort": "data/s41598-020-73558-3_sepsis_survival_study_cohort.csv",
    "validation_cohort": "data/s41598-020-73558-3_sepsis_survival_validation_cohort.csv",
}
dict_df = dict()
for key, file in file_names.items():
    dict_df[key] = pd.read_csv(file)

### **Dataset information**

#### **What do the instances in this dataset represent?**

For the primary cohort, they represent records of patients affected by sepsis potential preconditions (ante Sepsis-3 definition); for the study cohort, they represent only the patients’ admissions defined by the novel Sepsis-3 definition.

#### **Are there recommended data splits?**

No recommendation, standard train-test split could be used. Can use three-way holdout split (i.e., training, validation/development, testing) when doing model selection.

#### **Does the dataset contain data that might be considered sensitive in any way?**

Yes. It contains information about the gender and age of the patient.

#### **Was there any data preprocessing performed?**

All the categorical variables have been encoded (so no preprocessing is necessary).

#### **Additional Information**

Primary cohort from Norway:
- 4 features for 110,204 patient admissions
- file: 's41598-020-73558-3_sepsis_survival_primary_cohort.csv'

Study cohort (a subset of the primary cohort) from Norway:
- 4 features for 19,051 patient admissions
- file: 's41598-020-73558-3_sepsis_survival_study_cohort.csv'

Validation cohort from South Korea:
- 4 features for 137 patients
- file: 's41598-020-73558-3_sepsis_survival_validation_cohort.csv'

The validation cohort from South Korea was used by Chicco and Jurman (2020) as an external validation cohort to confirm the generalizability of their proposed approach. 

#### **Has Missing Values?**

No

# I. Despriction des datasets
### 1. Nombre d'échantillons et de personnes décédées (target)

In [29]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

cohorts = ["primary_cohort", "study_cohort", "validation_cohort"]
sizes = [len(dict_df[cohort]) for cohort in cohorts]

fig = make_subplots(rows=2, cols=1, subplot_titles=[
    "Nombre d'échantillons", "Pourcentage de morts"
])

fig.add_trace(
    go.Bar(x=cohorts, y=sizes, text=sizes, textposition="auto", name="Taille"),
    row=1, col=1
)

prop_deads = [
    100 * (len(dict_df[cohort]) - dict_df[cohort].hospital_outcome_1alive_0dead.sum()) / len(dict_df[cohort])
    for cohort in cohorts
]

fig.add_trace(
    go.Bar(x=cohorts, y=prop_deads, text=[f"{v:.1f}%" for v in prop_deads],
           textposition="auto", name="% morts"),
    row=2, col=1
)

fig.update_layout(
    title="Nombre de patients et de morts",
    height=700,
    showlegend=False
)

fig.show()

On remarque bien que le dataset `study_cohort` est une sous partie de `primary_cohort`, représentant environ un cinquième des observations. \
D'ailleurs, dans ce dataset, le nombre de morts est bien plus important que dans `primary_cohort`. \
Le dataset de validation `validation_cohort`(provenant de données en Corée du Sud), est de taille biens moindre mais présente une proportion de patients n'ayant pas survécus similaires au dataset d'étude `study_cohort`.


### 2. Répartition des ages

In [30]:
fig = make_subplots(rows=2, cols=3, subplot_titles=cohorts)

for i, cohort in enumerate(cohorts):
    fig.add_trace(
        go.Histogram(x=dict_df[cohort].age_years, name=cohort, opacity=0.6),
        row=1, col=i+1
    )


fig.update_layout(
    title="Etude des âges",
    showlegend=False,
    height=400,
    width=900
)

fig.show()

fig = go.Figure()
for cohort in cohorts:
    fig.add_trace(
        go.Box(x=dict_df[cohort].age_years, name=cohort, opacity=0.6),

    )


fig.update_layout(barmode='group')  # ✅ côte à côte
fig.show()

`study_cohort` compte bien moins de patients jeunes (age < 40 ans) que les deux autres datasets. Cela a pour effet de décaler les moyenne et médianes d'age de `study_cohort` vers la droite.\
Mis à part cela, les répartitions d'age sont similaires, avec une mode aux alentours de 82 ans pour les deux populations. \
Cependant, le dataset de validation provenant des données coréennes ne contient que très peu d'ages aussi élevés. On note le 3e quantile à 72 ans. 
Ces nuances entre dataset d'entraînement et de test pourra poser problème à l'établissement du modèle final. 


### 3. Répartition des sexes

In [31]:
fig = make_subplots(rows=1, cols=len(cohorts), subplot_titles=cohorts)

for i, cohort in enumerate(cohorts):
    fig.add_trace(
        go.Histogram(x=dict_df[cohort].sex_0male_1female, name=cohort, xbins=dict(start=-0.5, end=1.5, size=1), opacity=0.6),
        row=1, col=i+1
    )


fig.update_layout(
    title="Nombre d'hommes (0) vs nombre de femmes (1)",
    showlegend=False,
    height=400,
    width=900,
    bargap=0.3
)

fig.show()

On remarque plus d'hommes que de femmes dans chaque population étudiée \
Cependant, pour les données coréennes il y a seulement 47 pour 90 hommes : elles ne représentent qu'un tiers des patients étudiés ici. \
Pour le groupe primaire il y a 47% de femmes et pour le groupe d'étude il y en a 45%, c'est similaire. \




### 4. Vérification age et sexe indépendant

In [33]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=1,
    cols=len(cohorts),
    subplot_titles=cohorts)


colors = {0: '#1f77b4', 1: '#ff7f0e'}  # bleu = homme, orange = femme

for i, cohort in enumerate(cohorts):
    fig.add_trace(
        go.Histogram(
            x=dict_df[cohort].loc[dict_df[cohort].sex_0male_1female == 0, 'age_years'],
            name='Homme',
            opacity=0.6,
            marker_color=colors[0],
            nbinsx=20
        ),
        row=1, col=i+1
    )

    fig.add_trace(
        go.Histogram(
            x=dict_df[cohort].loc[dict_df[cohort].sex_0male_1female == 1, 'age_years'],
            name='Femme',
            opacity=0.6,
            marker_color=colors[1],
            nbinsx=20
        ),
        row=1, col=i+1
    )

fig.update_layout(
    title="Répartition des âges par sexe selon la cohorte",
    xaxis_title="Âge",
    yaxis_title="Nombre de patients",
    barmode='overlay',   
    height=500,
    width=1200,
    showlegend=True
)

fig.show()


### 5. Effets généraux de l'age et du sexe sur la probabilité de mort

In [38]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# --- Création de la figure avec axes Y partagés ---
fig = make_subplots(
    rows=1,
    cols=len(cohorts),
    subplot_titles=list(cohorts),
    shared_yaxes=True  # ✅ même échelle Y pour tous les sous-graphes
)

# Couleurs par sexe
colors = {0: '#1f77b4', 1: '#ff7f0e'}

# Boucle sur les cohortes
for i, cohort in enumerate(cohorts, start=1):
    
    # Création de tranches d’âge (ex: tous les 10 ans)
    dict_df[cohort]['age_bin'] = pd.cut(dict_df[cohort].age_years, bins=range(0, 101, 10), right=False)

    # Calcul de la probabilité de mort (1 - taux de survie) par sexe et tranche d’âge
    prob = (
        dict_df[cohort]
        .groupby(['age_bin', 'sex_0male_1female'])
        .hospital_outcome_1alive_0dead
        .mean()
        .reset_index()
    )
    prob['death_prob'] = 1 - prob.hospital_outcome_1alive_0dead  # car 0 = mort, 1 = vivant

    # Tracer pour chaque sexe
    for sex in [0, 1]:
        subset = prob[prob['sex_0male_1female'] == sex]
        fig.add_trace(
            go.Scatter(
                x=subset['age_bin'].astype(str),
                y=subset['death_prob'],
                mode='lines+markers',
                name=f"{'Homme' if sex==0 else 'Femme'} ({cohort})",
                line=dict(color=colors[sex]),
                showlegend=(i == 1)  # ✅ la légende n’apparaît qu’une fois
            ),
            row=1, col=i
        )

# --- Mise en page générale ---
fig.update_layout(
    title="Visualisation des effets de l'âge et du sexe sur la probabilité de mort",
    height=500,
    width=1200,
    xaxis_title="Tranches d'âge (ans)",
    yaxis_title="Probabilité de mort",
    yaxis=dict(range=[0, 1]),  # ✅ même échelle [0,1] sur tous les subplots
    legend_title="Sexe",
    template="plotly_white",
    font=dict(size=13)
)

fig.show()


/tmp/ipykernel_1367/3974294505.py:25: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/tmp/ipykernel_1367/3974294505.py:25: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/tmp/ipykernel_1367/3974294505.py:25: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



### 6. Effets du nombre d'épisodes de sepsis sur la mortalité

# II. Création d'un modèle de classification

### 1. Choix du modèle à entraîner

##### Remarque : **pas de feature engineering à faire**

### 2. Dimensionnement de l'hyper paramètre en accord avec l'étude

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

/home/guillaumereynier/anaconda3/envs/myenv/lib/python3.12/site-packages/xgboost/core.py:377: FutureWarning:

Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc >= 2.28) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.



In [ ]:
Xtrain = dict_df["study_cohort"].drop(columns=["hospital_outcome_1alive_0dead"])
ytrain = dict_df["study_cohort"]["hospital_outcome_1alive_0dead"]

model = xgb.XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",  # ou "auc" si tu préfères
    use_label_encoder=False,
    n_estimators=200,
    learning_rate=0.05,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
)

# === 5. Entraînement ===
model.fit(Xtrain, ytrain)

X_val_primary = dict_df["primary_cohort"].drop(columns=["hospital_outcome_1alive_0dead"])
X_val_test = dict_df["study_cohort"].drop(columns=["hospital_outcome_1alive_0dead"])
y_val_primary = dict_df["primary_cohort"]["hospital_outcome_1alive_0dead"]
y_val_test = dict_df["study_cohort"]["hospital_outcome_1alive_0dead"]

# === 6. Prédiction et évaluation ===
y_pred_primary = model.predict(X_val_primary)
y_pred_test= model.predict(X_val_test)

print("Accuracy :", accuracy_score(y_val_primary, y_pred_primary))
print("\nRapport de classification primary :\n", classification_report(y_val_primary, y_pred_primary))
print("Accuracy :", accuracy_score(y_val_test, y_pred_test))

print("\nRapport de classification test :\n", classification_report(y_val_test, y_pred_test))



ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, the experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:age_bin: category

# III. Conclusion de l'étude : performance du modèle et limites